# Wildfire Detection Pipeline

In [1]:
import datetime
import json
import os

import matplotlib.pyplot as plt

from wildfire import goes, threshold_model, wildfire

WILDFIRE_FILENAME = "wildfires_{satellite}_{region}_s{start}_e{end}_c{created}.json"
DATETIME_FORMAT = "%Y-%m-%dT%H:%M:%S"

In [2]:
# Update these 

# GOES_DIRECTORY = "/nex/datapool/goes16"
GOES_DIRECTORY = "/Users/samuelzimmerman/Desktop/test_goes_dir"
WILDFIRE_DIRECTORY = "temporary_output"
SATELLITE = "noaa-goes17"
REGION = "M1"
START_TIME = datetime.datetime(2019, 10, 27, 10)
END_TIME = datetime.datetime(2019, 10, 27, 11)

In [3]:
# # uncomment, if you need to download data locally

local_filepaths = goes.downloader.download_files(
    local_directory=GOES_DIRECTORY,
    satellite=SATELLITE,
    region=REGION,
    start_time=START_TIME,
    end_time=END_TIME
)
print(local_filepaths)

_download_file_mp: 100%|██████████| 960/960 [00:47<00:00, 20.04it/s]


['/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001000275_e20193001000332_c20193001000375.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001001275_e20193001001332_c20193001001381.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001002275_e20193001002332_c20193001002380.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001003275_e20193001003332_c20193001003378.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001004275_e20193001004332_c20193001004382.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01_G17_s20193001005275_e20193001005332_c20193001005377.nc', '/Users/samuelzimmerman/Desktop/test_goes_dir/ABI-L1b-RadM/2019/300/10/OR_ABI-L1b-RadM1-M6C01

In [5]:
def persist_wildfires(local_directory, wildfire_directory, satellite, region, start, end):
    local_filepaths = goes.utilities.list_local_files(
        local_directory=local_directory,
        satellite=satellite,
        region=region,
        start_time=start,
        end_time=end
    )

    wildfires = wildfire.find_wildfires_goes(filepaths=local_filepaths)
    print(len(wildfires))

    wildfires_filepath = os.path.join(
        wildfire_directory,
        WILDFIRE_FILENAME.format(
            satellite=satellite,
            region=region,
            start=start.strftime(DATETIME_FORMAT),
            end=end.strftime(DATETIME_FORMAT),
            created=datetime.datetime.utcnow().strftime(DATETIME_FORMAT),
        ),
    )
    print("Persisting wildfires to %s".format(wildfires_filepath))
    with open(wildfires_filepath, "w+") as buffer:
        json.dump(dict(enumerate(wildfires)), buffer)
    return wildfires_filepath, local_filepaths


def get_non_wildfire_filepaths(filepaths, wildfires):
    wildfire_scan_times = [
        datetime.datetime.strptime(wildfire_scan["scan_time_utc"], DATETIME_FORMAT) 
        for wildfire_scan in wildfires
    ]
    return [
        filepath for filepath in filepaths
        if goes.utilities.parse_filename(filepath)[3] not in wildfire_scan_times
    ]

## pipeline

In [6]:
wildfires_filepath, all_local_files  = persist_wildfires(
    local_directory=GOES_DIRECTORY,
    wildfire_directory=WILDFIRE_DIRECTORY,
    satellite=SATELLITE,
    region=REGION,
    start=START_TIME,
    end=END_TIME
)

with open(wildfires_filepath, "r") as buff:
    wildfires = json.load(buff)

print(len(wildfires))

parse_scan_for_wildfire: 100%|██████████| 60/60 [00:51<00:00,  1.16it/s]

60
Persisting wildfires to %s
60


## Get non wildfires

In [8]:
non_wildfire_filepaths = get_non_wildfire_filepaths(filepaths=all_local_files, wildfires=wildfires)
# get some of these as scans?
# goes.read_netcdfs(local_filepaths=non_wildfire_filepaths[:100])  ???

TypeError: string indices must be integers

## Get wildfire scans

In [11]:
# get scans somehow??

# # useful methods for getting scans

def satellite_helper(short_form):
    if short_form == "G17":
        satellite = "noaa-goes17"
    elif short_form == "G16":
        satellite = "noaa-goes16"     
    return satellite


def get_goes_scans_with_wildfires():
    with open(wildfires_filepath) as f:
        wildfire_scan_dict = json.load(f)
        wildfire_scans = []
        for key, value in wildfire_scan_dict.items():
            satellite = satellite_helper(value['satellite'])
            scan = goes.get_goes_scan(
                satellite=satellite, 
                region=value['region'], 
                scan_time_utc= datetime.datetime.strptime(value['scan_time_utc'][:-6],'%Y-%m-%dT%H:%M:%S'),
                local_directory=local_filepaths,
                s3=False
            )
            wildfire_scans.append(scan)
    print(wildfire_scans)
    return wildfire_scans

def plot_wildfire_scans(wildfire_scans):
    for scan in wildfire_scanes:
        print("Plotting All Bands for Wildfire")
        scan.plot();
        print("Plotting Wildfire Pixels for Scan")
        wildfire.plot_wildfires_goes(goes_scan=scan);
        
        model_features = wildfire.get_model_features_goes(goes_scan=scan)
        _, axes = plt.subplots(ncols=2, nrows=2, figsize=(10, 10))
        axes = np.ravel(axes)
        for axis, (feature_name, feature_value) in zip(axes, model_features._asdict().items()):
            axis.imshow(feature_value)
            axis.set_title(feature_name)
        
            
get_goes_scans_with_wildfires()



# goes.read_netcdfs(local_filepaths=???)

# (1) Get GOES SCAN objects from JSON 
# (2) Plot/inspect GOES SCAN
# (3) Wildfire Object Sketch

<class 'datetime.datetime'>






glob:   0%|          | 0/1 [00:00<?, ?it/s]

RecursionError: maximum recursion depth exceeded while calling a Python object

## Model prediction on wildfires

do scans that model says have wildfires actually have wildfires? visually inspect?

`wildfire.plot_wildfires_goes(goes_scan=???)` 

In [ ]:
# run through model

# # useful methods on a GoesScan object

# wildfire.predict_wildfires_goes(goes_scan=???)
# wildfire.plot_wildfires_goes(goes_scan=???)
# goes_scan.plot(bands=list(range(1, 17)))  # where `goes_scan` is a GoesScan object

In [ ]:
## some type of scoring? or validation?